In [ ]:
# Exectue from the vetting folder


import pandas as pd
import pyam
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
pio.templates
import os
# os.chdir('C:\\Github\\eu-climate-advisory-board-workflow\\vetting')
from vetting_functions import *



In [ ]:
pio.templates
pio.templates.default = 'seaborn'

In [ ]:
instance = 'eu-climate-advisory-board'
# vstr = '20230329'
main_folder = f'outputs\\'
years = list(range(2015,2071))


In [ ]:

df = pyam.read_iiasa(instance,
                     year=years,
                     region='EU27', 
                     meta=True)


# Select either of the below
# outputs = 'plots_C1_excludeFalse'
outputs = 'plots_C1_excludeTrue'


output_folder = f'{main_folder}iconics\\'
if not os.path.exists(f'{main_folder}iconics\\'):
    os.makedirs(f'{main_folder}iconics\\')
    print("created folder : ", f'{main_folder}iconics\\')

os.chdir(output_folder)

poles = df.filter(model='POLES*')

if outputs == 'plots_C1_excludeFalse':
    df.filter(Category=['C1*','Regional only'], keep=True, inplace=True)
    df.filter(exclude=False, keep=True, inplace=True)
    df.filter(model='POLES*', keep=False, inplace=True)
    df.append(poles, inplace=True)
elif outputs == 'plots_C1_excludeTrue':
    df.filter(Category=['C1*','Regional only'], keep=True, inplace=True)
    df.filter(model='POLES*', keep=False, inplace=True)
    df.append(poles, inplace=True)


dfmeta = df.meta.copy().reset_index()
dfmeta['mskey'] = dfmeta['model'].str[:4]+'>>'+dfmeta['scenario']
dfmeta['model_stripped'] = dfmeta['model'].apply(strip_version).values
dfp = df.as_pandas()
dfp['mskey'] = dfp['model'].str[:4]+'>>'+dfp['scenario']

In [ ]:
prefix = 'Diagnostics|Harmonized|'

ghg_var = f'{prefix}Emissions|Kyoto Gases (AR4) (EEA - intra-EU only)'

dfp.sort_values(by=['model','scenario','variable','year'], axis=0, inplace=True)
dfp.loc[(dfp.variable==ghg_var),['model','scenario','variable','year','value'] ]

In [ ]:
df

In [ ]:
def plot_line(dfp, variable, output='fig_data', x='year', y='value', color='model', line_group='mskey', hover_name='scenario', years=[]):
    if len(years)>0:
        dfpin = dfp.loc[(dfp.variable==variable) & dfp.year.isin(years)]
    else:
        dfpin = dfp.loc[dfp.variable==variable]
    dfpins = dfpin.sort_values(by=['model','scenario','year'], axis=0, ignore_index=True)
#     dfpins.head()

    if output=='fig_data':
        fig = px.line(dfpins, x=x, y=y, title=variable,  color=color, line_group=line_group, hover_name=hover_name)
        fig.show()
        return fig, dfpins
    elif output=='subplot':
        return px.line(dfpins, x=x, y=y, title=variable,  color=color, line_group=line_group, hover_name=hover_name)




## Emissions

In [ ]:
fig, dfpins = plot_line(dfp, variable=ghg_var)#, years=range(2020,2051))

In [ ]:
dfpins = plot_line(dfp, variable=ghg_var)

In [ ]:
dfpins = plot_line(dfp, variable=f'{prefix}Emissions|CO2')

## Primary Energy Renewables

In [ ]:
dfpins = plot_line(dfp, variable=f'Primary Energy|Renewables (incl.Biomass)|Share', years=range(2020,2051))

In [ ]:
dfpins = plot_line(dfp, variable=f'Primary Energy|Non-Biomass Renewables|Share')

## Secondary Energy|Electricity Renewables

In [ ]:
dfpins = plot_line(dfp, variable=f'Secondary Energy|Electricity|Renewables (incl.Biomass)|Share')#, years=range(2020,2051))

In [ ]:
dfpins = plot_line(dfp, variable=f'Secondary Energy|Electricity|Non-Biomass Renewables|Share')

## Final Energy|Electrification

In [ ]:
dfpins = plot_line(dfp, variable='Final Energy|Electrification|Share')

In [ ]:
def plot_scatter_meta(df,  x='', y='', markersize=20, kwargs={}):
    #color='model', hover_name='scenario', title=''
    # Input df as df.meta table
#     dfpins = dfpin.sort_values(by=['model','scenario','year'], axis=0, ignore_index=True)
#     dfpins.head()
    df = df[['model','scenario',x ,y]]
    fig = px.scatter(df, x=x, y=y, **kwargs) #title=title,  color=color, hover_name=hover_name)
    fig.update_traces(marker=dict(size=markersize,
                          line=dict(width=2,
                                    color='DarkSlateGrey')),
              selector=dict(mode='markers'))
    fig.show()
    return fig

def plot_scatter_variables(dfp, markersize=20,  x='', y='', year=2050, kwargs={}):
    ms= ['model','scenario']
    xd = dfp.query("variable==@x and year==@year")
    yd = dfp.query("variable==@y and year==@year")
    xd = xd.rename(columns={'value':x})
    yd = yd.rename(columns={'value':y})
    xd = xd[['model','scenario',x]]
    yd = yd[['model','scenario',y]]
    data = pd.merge(xd, yd, on=ms)

    fig = px.scatter(data, x=x, y=y, **kwargs)
    fig.update_traces(marker=dict(size=markersize,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
    fig.show()
    return fig

In [ ]:
dfmeta.columns

# Scatter plots

In [ ]:
year = 2050
config = {'FE_x_PE_import': dict(x='Final Energy',
                               y='PE Import dependency',
                               title=f'<b>Final energy vs PE Import dependency in {year}</b>',
                               ptype='vari'),
          
          'FEelec_x_PERenewNB': dict(x='Final Energy|Electrification|Share',
                               y='Primary Energy|Non-biomass renewables|Share',
                               title=f'<b>Final En. Electrification vs Renewables share in {year}</b>',
                               ptype='vari'),
          
          'FE_x_PERenewNB': dict(x='Final Energy',
                               y='Primary Energy|Non-biomass renewables|Share',
                               title=f'<b>Final energy vs Renewables share in {year}</b>',
                               ptype='vari'),
          'FE_x_cumBECCS': dict(x=f'Final Energy in {year}, EJ/yr',
                               y='cumulative BECCS (2020-2050, Gt CO2)',
                               title=f'<b>Final energy vs Cum BECCS in {year}</b>',          
                               ptype='meta'),
          
          'FE_x_Hydrogen': dict(x=f'Final Energy in {year}, EJ/yr',
                               y=f'Hydrogen production|Final Energy|Share in {year}, %',
                               title=f'<b>Final energy vs Hydrogen prod. {year}</b>',          
                               ptype='meta'),          
          
          'FEelec_x_Hydrogen': dict(x=f'Final Energy|Electrification|Share in {year}, %',
                               y=f'Hydrogen production|Final Energy|Share in {year}, %',
                               title=f'<b>Final En. Electrification vs Hydrogen prod. {year}</b>',          
                               ptype='meta'),           
          'Hydrogen_x_PERenewNB': dict(x=f'Hydrogen production|Final Energy|Share in {year}, %',
                               y=f'Primary Energy|Non-biomass renewables|Share in {year}, %',
                               title=f'<b>Hydrogen prod. vs Renewables {year}</b>',          
                               ptype='meta'), 
          'Hydrogen_x_PEImportDep': dict(x=f'Hydrogen production|Final Energy|Share in {year}, %',
                               y=f'PE Import dependency in 2050, %',
                               title=f'<b>Hydrogen prod. vs PE import dependency {year}</b>',          
                               ptype='meta'),           
          'Hydrogen_x_PEFossilImportDep': dict(x=f'Hydrogen production|Final Energy|Share in {year}, %',
                               y=f'PE Import dependency|Fossil in 2050, %',
                               title=f'<b>Hydrogen prod. vs PE Fossil import dependency {year}</b>',          
                               ptype='meta'),               
          'PERenewNB_x_PEFossilImportDep': dict(x='Primary Energy|Non-biomass renewables|Share in 2050, %',
                               y=f'PE Import dependency|Fossil in 2050, %',
                               title=f'<b>Renewables. vs PE Fossil import dependency {year}</b>',          
                               ptype='meta'),         
        'cumBECCS_x_NonCO2': dict(x='Emissions|Total Non-CO2 in 2050, Mt CO2-equiv/yr',
                               y='cumulative BECCS (2020-2050, Gt CO2)',
                               title=f'<b>Non-CO2 emissions in 2050 vs Cum BECCS {year}</b>',          
                               ptype='meta'),   
        'cumGHGs_x_cumCO2': dict(x='cumulative GHGs (incl. indirect AFOLU) (2020-2050, Gt CO2-equiv)',
                               y='cumulative net CO2 (2020-2050, Gt CO2)',
                               title=f'<b>Cum. GHGs vs Cum. net CO2 in 2050</b>',          
                               ptype='meta'), 
        'cumCO2_x_NonCO2': dict(x='cumulative net CO2 (2020-2050, Gt CO2)',
                               y='Emissions|Total Non-CO2 in 2050, Mt CO2-equiv/yr',
                               title=f'<b>cum. CO2 emissions vs Non-CO2 in {year}</b>',          
                               ptype='meta'),    
        'GHGsred1930_x_cumGHG3050': dict(x='GHG emissions reductions 2019-2030 %',
                               y='cumulative GHGs (incl. indirect AFOLU) (2030-2050, Gt CO2-equiv)',
                               title=f'<b>GHG reductions 2019-2030 vs Cum. GHGs 2030-50</b>',          
                               ptype='meta'), 
        'GHGsred1930_x_cumGHG2030': dict(x='GHG emissions reductions 2019-2030 %',
                               y='cumulative GHGs (incl. indirect AFOLU) (2020-2030, Gt CO2-equiv)',
                               title=f'<b>GHG reductions 2019-2030 vs Cum. GHGs 2020-30</b>',          
                               ptype='meta'),      
         }
          
          
# dout, fig = plot_scatter_variables(dfp, x=x, y=y, year=year, kwargs=akwargs)

In [ ]:
output_folder

In [ ]:
os.getcwd()

In [ ]:
year = 2050

for scat, info in config.items():
    akwargs = {'color':'model', 'hover_name':'scenario', 'title':info['title']}

    
    if info['ptype']=='vari':
#         x = info['x']
        fig = plot_scatter_variables(dfp, x=info['x'], y=info['y'], year=year, kwargs=akwargs)
    
    elif info['ptype']=='meta':
        cols = ['model','scenario', info['x'], info['y'], 'year']

        fig = plot_scatter_meta(dfp[cols].query("year==@year"), x=info['x'], y=info['y'], kwargs=akwargs)

    filename = f'scatter_{scat}'
    fig.update_layout(width=None).write_html(filename+'.html', include_plotlyjs='cdn')
    fig.write_image(filename+'.png')


## Year net zero scatters

In [ ]:
# x = 'Final Energy|Electrification|Share in year of net zero, %'
# y = 'Primary Energy|Non-biomass renewables|Share in year of net zero, %'

# akwargs = {'color':'model', 'hover_name':'scenario',
#            'title':'<b>Year NZ: Final energy electrification vs Renewables share</b>', 'size_max': 20}

# fig = plot_scatter_meta(dfp, x=x, y=y, kwargs=akwargs) 

In [ ]:
# x = 'Final Energy in year of net zero, EJ/yr'
# y = 'cumulative BECCS (2020 to year of net zero CO2, Gt CO2)'
# akwargs = {'color':'model', 'hover_name':'scenario',
#            'title':'<b>Year NZ: Final energy vs Cum BECCS</b>', 'size_max': 20}

# fig = plot_scatter_meta(dfp, x=x, y=y, kwargs=akwargs) 

# Parallel coordinates

In [ ]:
# Set numbering for models
modelnums = {v:k for k,v in enumerate(dfmeta.model.unique())}
modelnums = dict(reversed(modelnums.items()))
dfmeta['modelnum'] = dfmeta.model.replace(modelnums)
mns = [f'{k}={v}' for k,v in modelnums.items()]
mns = '<br>'.join(mns)
modtxt = f'Model<br>{mns}'

In [ ]:
# define plotting function
def plot_parc(dfmeta, dimensions, colorcol='modelnum', title='', filename='x.html', colorscale='spectral'):
    fig = go.Figure(data=
    go.Parcoords(
        line = dict(color = dfmeta[colorcol],
                   colorscale = colorscale,
                   showscale = True,
#                    cmin = -4000,
#                    cmax = -100
                   ),
        dimensions = dimensions,
        tickfont = dict(size=14),
        labelfont = dict(size=18, ),
#         labelangle = -45,
        labelside='bottom',
        name='dfddfd'#dfmeta['mskey']
#                               unselected = dict(line = dict(color = 'green', opacity = 0.5))

    )
)

    mns = [f'{k}={v}' for k,v in modelnums.items()]
    mns = ', '.join(mns)
    title = title
    fig.update_layout(
        plot_bgcolor = 'white',
        paper_bgcolor = 'white',
        title = title,
        margin=dict(l=80, r=50, t=150, b=200)

    #     yaxis = dict( tickfont = dict(size=50))
    )
    fig.show()
    fig.update_layout(width=None).write_html(filename, include_plotlyjs='cdn')



## Renewables and final energy

In [ ]:
title = f'<b>Renewables & final energy</b>'
pcspec = {
        'Cum CO2 to YNZ<br>Gt CO2' : {'col': 'cumulative net CO2 (2020 to year of net zero CO2, Gt CO2)',},
#         'GHGs in YNZ' : {'col':  'Emissions|Kyoto Gases (incl. indirect AFOLU) in year of net zero, Mt CO2-equiv/yr',},
#         'GHGs in 2050': {'col':'Emissions|Kyoto Gases (incl. indirect AFOLU) in 2050, Mt CO2-equiv/yr'},
        'Non-CO2 red. by 2050<br>%': {'col': 'Non-CO2 emissions reductions 2020-2050 %',},
        'PE NB Renew. share in YNZ<br>%'    : {'col': 'Primary Energy|Non-biomass renewables|Share in year of net zero, %',},
        'Final Energy in YNZ<br>EJ/yr' : {'col':   'Final Energy in year of net zero, EJ/yr',},
        'Electr. FE in YNZ<br>%' : {'col':   'Final Energy|Electrification|Share in year of net zero, %',},
        modtxt: {'col': 'modelnum'},
}
labels = {v['col']:k for k,v in pcspec.items()}
dimensions = list(dict(label=v, values=dfmeta[k]) for k,v in labels.items())
# dimensions[-1]['tickvals'] = list(modelnums.keys())

filename = f'PC_Renew_FE.html'
colorscale = ['red', 'blue','orange','green','magenta']
plot_parc(dfmeta, dimensions, colorcol='modelnum', title=title, filename=filename, colorscale=colorscale)

## BECCS & Fossil

In [ ]:
title = f'<b>BECCS & Fossil</b>'
pcspec = {
        'Cum CO2 to YNZ<br>Gt CO2' : {'col': 'cumulative net CO2 (2020 to year of net zero CO2, Gt CO2)',},
#         'GHGs in YNZ' : {'col':  'Emissions|Kyoto Gases (incl. indirect AFOLU) in year of net zero, Mt CO2-equiv/yr',},
#         'GHGs in 2050': {'col':'Emissions|Kyoto Gases (incl. indirect AFOLU) in 2050, Mt CO2-equiv/yr'},
        'Non-CO2 red. by 2050<br>%': {'col': 'Non-CO2 emissions reductions 2020-2050 %',},
        'PE NB Renew. share in YNZ<br>%'    : {'col':  'Primary Energy|Non-biomass renewables|Share in year of net zero, %',},
        'BECCS in YNZ<br>Mt CO2/yr' : {'col':   'Carbon Sequestration|CCS|Biomass in year of net zero, Mt CO2/yr',},
        'PE Fossil share in YNZ<br>%' : {'col':   'Primary Energy|Fossil|Share in year of net zero, %',},
        modtxt: {'col': 'modelnum'},
}
labels = {v['col']:k for k,v in pcspec.items()}
dimensions = list(dict(label=v, values=dfmeta[k]) for k,v in labels.items())
filename = f'PC_BECCS_Fossil.html'
plot_parc(dfmeta, dimensions, colorcol='modelnum', title=title, filename=filename, colorscale=colorscale)

## Emissions GHG /CO2 / Non-CO2 tradeoff

In [ ]:
title = f'<b>GHG /CO2 /  Non-CO2 tradeoffs</b>'
pcspec = {
        'year NZ CO2': {'col': 'year of net-zero CO2 emissions (threshold=0 Gt CO2/yr)'},
        'Cum CO2 to YNZ<br>Gt CO2' : {'col': 'cumulative net CO2 (2020 to year of net zero CO2, Gt CO2)',},
        'GHGs in YNZ<br>My CO2-equiv' : {'col':  'Emissions|Kyoto Gases (incl. indirect AFOLU) in year of net zero, Mt CO2-equiv/yr',},
        'Non-CO2 in YNZ<br>My CO2-equiv': {'col':'Emissions|Total Non-CO2 in year of net zero, Mt CO2-equiv/yr'},
        'Non-CO2 red. by 2050<br>%': {'col': 'Non-CO2 emissions reductions 2020-2050 %',},
        'BECCS in YNZ<br>Mt CO2/yr' : {'col':   'Carbon Sequestration|CCS|Biomass in year of net zero, Mt CO2/yr',},
        modtxt: {'col': 'modelnum'},
}
labels = {v['col']:k for k,v in pcspec.items()}
dimensions = list(dict(label=v, values=dfmeta[k]) for k,v in labels.items())
filename = f'PC_GHG_CO2_tradeoffs.html'
plot_parc(dfmeta, dimensions, colorcol='modelnum', title=title, filename=filename, colorscale=colorscale)

## HTML histograms

In [ ]:

def plot_histo(dfo, columns, filename=None, color_column='model', nbins=25, title=''):
    
    fig = make_subplots(rows=len(columns), cols=1)

    colormap = dict(zip(dfo[color_column].unique(), px.colors.qualitative.Light24))
    in_legend = []
    threshold_plot = 0.5

    for i, column in enumerate(columns[4:]):


        # Create a histogram figure, and add all its traces to the main figure
        # Set bingroup to None to avoid sharing the same bins accross subplots
        for trace in px.histogram(dfo, x=column, color=color_column, nbins=nbins, color_discrete_map=colormap).data:
            fig.add_trace(trace.update(
                bingroup=None, showlegend=trace.showlegend & (i==0)
            ), col=1, row=i+1)


        # Give xaxis its title and change tickformat if necessary
        fig.update_xaxes(
            row=i+1,
            title=column, title_standoff=0,
#             tickformat='%' if '%' in column else None
        )

    
    fig.update_yaxes(title='# scenarios', title_standoff=0)
    fig.update_layout(
        height=200*len(columns), width=900,
        barmode='stack',
        margin={'l': 50, 'r': 20, 't': 40, 'b': 60},
        legend_y=0.5,
        title=title,
        font=dict(size=18)
    )

##%% Save to html
    if filename==None:
        fig.show()
    else:
        fig.update_layout(width=None).write_html(filename, include_plotlyjs='cdn')



## Emissions

In [ ]:
columns = ['model','scenario','model_stripped', 'mskey',
        'year of net-zero CO2 emissions (threshold=0 Gt CO2/yr)',
        'year of net-zero GHGs full emissions (threshold=0 Gt CO2-equiv/yr)',
        'cumulative net CO2 (2020 to year of net zero CO2, Gt CO2)',
        'cumulative net CO2 (2020-2050, Gt CO2)',
        'cumulative Non-CO2 (2020-2050, Gt CO2-equiv)',
        'cumulative CCS (2020-2050, Gt CO2)',
        'cumulative BECCS (2020-2050, Gt CO2)',
        'cumulative GHGs (incl. indirect AFOLU) (2020-2050, Gt CO2-equiv)',
        'Non-CO2 emissions reductions 2020-2050 %',
        'Emissions|CO2 in 2050, Mt CO2/yr',
        'Emissions|Total Non-CO2 in 2050, Mt CO2-equiv/yr',
        'Emissions|Kyoto Gases (incl. indirect AFOLU) in year of net zero, Mt CO2-equiv/yr',
        'Emissions|Kyoto Gases (incl. indirect AFOLU) in 2050, Mt CO2-equiv/yr',
        'GHG emissions reductions 1990-2035 %',
           'GHG emissions reductions 1990-2040 %'
#         'Emissions|Kyoto Gases in 2050, Mt CO2-equiv/yr',
]
title = '<b>Emissions</b>'
filename = f'Histo_emissions.html'
dfo = dfmeta[columns]
plot_histo(dfo, columns, filename=filename, color_column='mskey', nbins=25, title=title)

## Characteristics

In [ ]:
columns = ['model','scenario','model_stripped', 'mskey',
        'year of net-zero CO2 emissions (threshold=0 Gt CO2/yr)',
        'year of net-zero GHGs full emissions (threshold=0 Gt CO2-equiv/yr)',
       'Emissions|Total Non-CO2 in 2050, Mt CO2-equiv/yr',
#        'Emissions|CO2|AFOLU in 2050, Mt CO2/yr',
       'Carbon Sequestration|CCS|Biomass in 2050, Mt CO2/yr',
       'Primary Energy|Renewables (incl.Biomass)|Share in 2050, %',
       'Primary Energy|Non-biomass renewables|Share in 2050, %',
       'Primary Energy|Fossil|Share in 2050, %',
       'Primary Energy|Fossil|w/o CCS|Share in 2050, %',
       'Final Energy in 2050, EJ/yr',
       'Final Energy|Electrification|Share in 2050, %',
       'PE Import dependency in 2050, %',
       'PE Import dependency|Fossil in 2050, %',
       'Trade in 2050, EJ/yr',
       'Trade|Primary Energy|Fossil in 2050, EJ/yr', 
]

title = '<b>Energy characteristics</b>'
filename = f'Histo_energy.html'
dfo = dfmeta[columns]
plot_histo(dfo, columns, filename=filename, color_column='mskey', nbins=25, title=title)

## Sectoral

In [ ]:
columns = ['model','scenario','model_stripped', 'mskey',
       'Primary Energy|Renewables (incl.Biomass)|Share in 2050, %',
       'Primary Energy|Non-biomass renewables|Share in 2050, %',
       'Primary Energy|Fossil|Share in 2050, %',
       'Primary Energy|Fossil|w/o CCS|Share in 2050, %',
       'Final Energy in 2050, EJ/yr',
       'Final Energy|Electrification|Share in 2050, %',
       'Final Energy|Industry|Fossil|Share in 2050, %',
       'Final Energy|Residential and Commercial|Fossil|Share in 2050, %',
       'Final Energy|Transportation|Fossil|Share in 2050, %',
       'Carbon Sequestration|CCS in 2050, Mt CO2/yr',
       'Carbon Sequestration|CCS|Biomass in 2050, Mt CO2/yr',
       'Carbon Sequestration|CCS|Fossil in 2050, Mt CO2/yr',
       'Carbon Sequestration|CCS|Industrial Processes in 2050, Mt CO2/yr',
       'Trade|Primary Energy|Fossil in 2050, EJ/yr',
       'PE Import dependency|Fossil in 2050, %',
]

title = '<b>Sectoral characteristics</b>'
filename = f'Histo_sectoral.html'
dfo = dfmeta[columns]
plot_histo(dfo, columns, filename=filename, color_column='mskey', nbins=25, title=title)